## Loading and preprocessing ISTAT data

Processing pipeline to collect Istat cpa 2011 data for selected cities from downloaded data dump.

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
from os import path, listdir

In [3]:
## TODO: find way to put this into some global settings
import os
import sys
# go up two levels in path
nb_dir = os.path.dirname(os.path.dirname(os.getcwd()))
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    sys.path.append(nb_dir+'/references')

import commonCfg

In [5]:
## Loading ISTAT data
fileList = commonCfg.get_istat_filelist()
dfList = []
dataDict = {}
for filename in fileList:
    regionData = pd.read_csv(
        path.join(commonCfg.cpaPath, filename), sep=';', encoding='latin').set_index('SEZ2011')
    # extract the councils we are intersted in
    for city in commonCfg.cityList:
        cityData = regionData[regionData.COMUNE==city]
        if cityData.size>0:
            dataDict[city] = cityData


['R17_indicatori_2011_sezioni.csv', 'R14_indicatori_2011_sezioni.csv', 'R12_indicatori_2011_sezioni.csv', 'R05_indicatori_2011_sezioni.csv', 'R08_indicatori_2011_sezioni.csv', 'R10_indicatori_2011_sezioni.csv', 'R04_indicatori_2011_sezioni.csv', 'R20_indicatori_2011_sezioni.csv', 'R03_indicatori_2011_sezioni.csv', 'R06_indicatori_2011_sezioni.csv', 'R18_indicatori_2011_sezioni.csv', 'R11_indicatori_2011_sezioni.csv', 'R16_indicatori_2011_sezioni.csv', 'R09_indicatori_2011_sezioni.csv', 'R07_indicatori_2011_sezioni.csv', 'R15_indicatori_2011_sezioni.csv', 'R02_indicatori_2011_sezioni.csv', 'R19_indicatori_2011_sezioni.csv', 'R13_indicatori_2011_sezioni.csv', 'R01_indicatori_2011_sezioni.csv']


In [ ]:
# Join sez geofile for Milano
sezNameCol = 'SEZ2011'
shapeDataMilano = gpd.read_file('../data/milanoSezGeo')
print(shapeDataMilano[sezNameCol].isin(dataDict['Milano'].index).mean())
print(dataDict['Milano'].index.isin(shapeDataMilano[sezNameCol]).mean())
joinedData = pd.merge(shapeDataMilano, dataDict['Milano'], how='inner', right_index=True, left_on=sezNameCol)
joinedData.SEZ2011 = joinedData.SEZ2011.astype(int)

In [ ]:
# collect quartiere label
quartiereLabels = pd.read_csv('../data/Milano_sezToQuartieri.csv')
# fix typo
quartiereLabels.NIL.replace(to_replace='MAGENTA - S.VITTORE', value='MAGENTA - S. VITTORE', inplace=True)

quartieriData = gpd.read_file('../data/Milano_quartieri.geojson')
# join
quartiereLabels = quartiereLabels.join(quartieriData[['NIL', 'ID_NIL']].set_index('NIL'), on='NIL')
quartiereLabels = quartiereLabels.set_index(sezNameCol)

assert not any(quartiereLabels.ID_NIL.isnull()), 'Typos in NIL field'

In [ ]:
joinedData = joinedData.join(quartiereLabels, on=sezNameCol)
joinedData = joinedData[~joinedData.ID_NIL.isnull()]
joinedData.ID_NIL = joinedData.ID_NIL.astype(int)
#joinedData.to_csv('test.csv')

In [ ]:
quartiereLabels

In [ ]:
# rename columns
joinedData.rename({'NIL':'quartiere', 'ID_NIL': 'IDquartiere'}, axis='columns', inplace=True)


In [ ]:
joinedData.to_file('../final/Milano_sezioni.geojson', driver='GeoJSON')

In [ ]:
zz = gpd.read_file('../final/Milano_sezioni.geojson')

In [ ]:
zz

In [ ]:
# export as csv to final folder
for city, cityData in dataDict.items():
    cityData.to_csv('../final/'+city+'_cpa_2011.csv', sep=';')